In [1]:
import os
from glob import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from torch.utils.data import Dataset, DataLoader

import numpy as np
from PIL import Image

In [2]:
seed = 1

lr = 0.001
momentum = 0.5

batch_size = 64
test_batch_size = 64

epochs = 1
no_cuda = False
log_interval = 100

# Model

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(5*5*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 5*5*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# Preprocess

In [4]:
os.listdir('../dataset/cifar/train')

['0_frog.png',
 '10000_automobile.png',
 '10001_frog.png',
 '10002_frog.png',
 '10003_ship.png',
 '10004_ship.png',
 '10005_cat.png',
 '10006_deer.png',
 '10007_frog.png',
 '10008_airplane.png',
 '10009_frog.png',
 '1000_truck.png',
 '10010_airplane.png',
 '10011_cat.png',
 '10012_frog.png',
 '10013_frog.png',
 '10014_dog.png',
 '10015_deer.png',
 '10016_ship.png',
 '10017_cat.png',
 '10018_bird.png',
 '10019_frog.png',
 '1001_deer.png',
 '10020_airplane.png',
 '10021_cat.png',
 '10022_automobile.png',
 '10023_deer.png',
 '10024_airplane.png',
 '10025_frog.png',
 '10026_frog.png',
 '10027_bird.png',
 '10028_horse.png',
 '10029_frog.png',
 '1002_cat.png',
 '10030_truck.png',
 '10031_airplane.png',
 '10032_deer.png',
 '10033_dog.png',
 '10034_horse.png',
 '10035_automobile.png',
 '10036_frog.png',
 '10037_horse.png',
 '10038_truck.png',
 '10039_automobile.png',
 '1003_bird.png',
 '10040_horse.png',
 '10041_horse.png',
 '10042_ship.png',
 '10043_airplane.png',
 '10044_cat.png',
 '10045_ho

In [5]:
train_paths = glob('../dataset/cifar/train/*.png')
test_paths = glob('../dataset/cifar/test/*.png')
len(train_paths), len(test_paths)

(50000, 10000)

참고: https://pytorch.org/tutorials/beginner/data_loading_tutorial.html#dataset-class

In [6]:
path = train_paths[0]

In [7]:
lbl_name = os.path.basename(path).split('_')[-1].replace('.png','')
lbl_name

'frog'

In [16]:
def get_label_name(path):
    lbl_name = os.path.basename(path).split('_')[-1].replace('.png','')
    return lbl_name

def get_label(lbl_name):
    label = np.argmax(classes == get_label(path))
    return label

In [17]:
img_pil = Image.open(path)
image = np.array(img_pil)

In [18]:
label_names = [get_label_name(path) for path in train_paths]
classes = np.unique(label_names)

In [19]:
image.shape

(32, 32, 3)

In [20]:
class Dataset(Dataset):
    def __init__(self, data_paths, transform=None):
        self.data_paths = data_paths
        self.transform = transform
    def __len__(self):
        return len(self.data_paths)

    def __getitem__(self, idx):
        path = self.data_paths[idx]
        # Read Image
        image = Image.open(path)
        
        # Get Label
        label = get_label(path)

        if self.transform:
            image = self.transform(image)

        return image, label

In [21]:
torch.manual_seed(seed)

use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [22]:
train_loader = torch.utils.data.DataLoader(
    Dataset(train_paths, 
            transforms.Compose([
                transforms.RandomHorizontalFlip(), 
                transforms.ToTensor(), 
                transforms.Normalize(
                    mean=[0.406], 
                    std=[0.225])])
           ),
    batch_size=batch_size, 
    shuffle=True, 
    **kwargs
)

test_loader = torch.utils.data.DataLoader(
    Dataset(test_paths, 
            transforms.Compose([
                transforms.ToTensor(), 
                transforms.Normalize(
                    mean=[0.406], 
                    std=[0.225])])
           ),
    batch_size=batch_size, 
    shuffle=False, 
    **kwargs
)


# Optimization

In [23]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

# Training

In [24]:
for epoch in range(1, epochs + 1):
    # Train Mode
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)  # https://pytorch.org/docs/stable/nn.html#nll-loss
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    
    # Test mode
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

RecursionError: maximum recursion depth exceeded

In [ ]:
# 왜오류가 뜰까...